# ZenQuotes

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import random
import time
import os

Alıntıları elde edeceğim URL'ini bir değişkene atıyorum. Pek çok farklı kategori olduğu için ben sadece seçtiğim konulardaki alıntıları çekmek istiyorum. Bunları da ```keywords``` listesine atıyorum.

In [4]:
url = "https://zenquotes.io/api/quotes/"

keywords = ["courage","dreams","fear","freedom","future","happiness","inspiration","leadership","life","love","past","success","work"]

Bu bağlantıya bir istek atmak için request paketini kullanıyorum.

In [ ]:
response = requests.get(url)

[Dokumantasyon](https://premium.zenquotes.io/zenquotes-documentation/)da yazdığı üzere bu istek bir JSON olarak dönecek. Bu yanıttan aldığım veriyi okuyorum ve ilk elemanını görüntülüyorum.

In [7]:
json_data = json.loads(response.text)
json_data[0]

{'q': 'I am not absentminded. It is the presence of mind that makes me unaware of everything else.',
 'a': 'Gilbert Chesterton',
 'c': '91',
 'h': '<blockquote>&ldquo;I am not absentminded. It is the presence of mind that makes me unaware of everything else.&rdquo; &mdash; <footer>Gilbert Chesterton</footer></blockquote>'}

Yine dokumantasyonuna bakınca buradaki yanıt:
- q : alıntı
- a : yazar (söyleyen kişi)
- c : karakter uzunluğu
- h : html formatındaki alıntı

Bu istekten dönen toplam alıntı sayısı:

In [8]:
len(json_data)

50

İstediğim kategorilerde alıntıları çekebilmek için bir fonksiyorum yazıyorum. Çektiğim her verideki söz sahibi kişiyi, alıntıyı ve kategorisini ayrı listelerde topluyorum. Bağlantı problemiyle karşılaştığım için fonksiyon her çağrıldığında 50 veriyi istiyorum. Üst üste çok istek gönderince red almamak için 8sn uyuma süresi tanımlıyorum.

In [15]:
def get_quotes():
    i = 0
    authors = []
    quotes = []
    keys = []
    keywords = ["courage","fear","leadership","love","success","work"]
    while i < 50:
        for keyword in keywords:
            link = "https://zenquotes.io/api/quotes/" + keyword
            response = requests.get(link)
            json_data = json.loads(response.text)
            i += 1
            rand = random.randint(0, len(json_data)-1)
            authors.append(json_data[rand]["a"])
            quotes.append(json_data[rand]["q"])
            keys.append(keyword)
            time.sleep(8)
    return authors, quotes, keys

In [41]:
%%time
a, q, k = get_quotes()

CPU times: total: 1.41 s
Wall time: 8min 12s


Çektiğim ve listelediğim verileri bir pandas dataframe haline getirip csv olarak kaydediyorum. Sorunsuz bir şekilde çektikten sonra hepsini tek bir dataframe haline getireceğim.

In [42]:
quotes = pd.DataFrame()
quotes["author"] = a
quotes["quote"] = q
quotes["keyword"] = k
quotes.head()

,author,quote,keyword
0,Elon Musk,"If life is a video game, the graphics are grea...",courage
1,Mae West,Those who are easily shocked should be shocked...,fear
2,Doug Ivester,Never let your memories be greater than your d...,leadership
3,Steve Maraboli,The right thing to do and the hard thing to do...,love
4,Christopher Reeve,You play the hand you're dealt. I think the ga...,success


In [43]:
quotes.to_csv("quotes_6.csv")

In [48]:
quotes = pd.DataFrame()
for file in os.listdir():
    i = 1
    if file.endswith(".csv"):
        df = pd.read_csv("quotes_" + str(i) + ".csv")
        i += 1
        quotes = pd.concat([quotes, df], axis=0)
quotes.sample(10)        

,Unnamed: 0,author,quote,keyword
53,53,Leo Tolstoy,Respect was invented to cover the empty place ...,work
9,9,Ming-Dao Deng,The wise accomplish all that they want without...,love
35,35,Tony Robbins,"If you can't, you must. If you must, you can.",work
21,21,Honore de Balzac,Every moment of happiness requires a great amo...,love
15,15,Alexandre Dumas,"I am not proud, but I am happy; and happiness ...",love
18,18,Hans Christian Andersen,"Life is like a beautiful melody, only the lyri...",courage
46,46,T.S. Eliot,"The journey, not the destination matters.",success
34,34,Voltaire,The mirror is a worthless invention. The only ...,success
49,49,Vincent van Gogh,"Great things are not done by impulse, but by a...",fear
39,39,Brian Tracy,Decide upon your major definite purpose in lif...,love


In [51]:
quotes.drop("Unnamed: 0", axis=1, inplace=True)
print(quotes.shape)
print("\n",quotes.info())

(324, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 324 entries, 0 to 53
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   324 non-null    object
 1   quote    324 non-null    object
 2   keyword  324 non-null    object
dtypes: object(3)
memory usage: 10.1+ KB

 None


Inspirational quotes provided by <a href="https://zenquotes.io/" target="_blank">ZenQuotes API</a>